In [150]:
import numpy as np
import pandas as pd
import json

# read and convert excel and csv to DataFrame
df_csv = pd.read_csv('UNSPSC_English.csv')
df_xlsx = pd.read_excel('Avalara_goods_and_services.xlsx')

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3194: DtypeWarning: Columns (1,2,5,8,9,12,15,16) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [151]:
# import nltk and download resources
import nltk
import ssl
try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    pass
else:
    ssl._create_default_https_context = _create_unverified_https_context
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /Users/mikec/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /Users/mikec/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [152]:
# parse the stop words and special characters out
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

stop_words = set(stopwords.words('english'))
stop_words.add(':')
stop_words.add('--')
stop_words.add('-')
stop_words.add('(')
stop_words.add(')')
stop_words.add('/')
stop_words.add('*')
stop_words.add(';')
stop_words.add('.')
stop_words.add('i.e')
stop_words.add('A')
stop_words.add('a')
stop_words.add(',')

In [153]:
def containsLetterAndNumber(input):
    has_letter = False
    has_number = False
    for x in input:
        if x.isalpha():
            has_letter = True
        elif x.isnumeric():
            has_number = True
        if has_letter and has_number:
            return True
    return False

In [154]:
def createParsedColumns(df, col1, col2, start_index, new_col):
    for index1, description1 in enumerate(df[col1], start_index):
        if not pd.isna(description1):
            word_tokens1 = word_tokenize(description1)

            filtered_sentence1 = []
            for w1 in word_tokens1:
                if not containsLetterAndNumber(w1) and w1 not in stop_words:
                    filtered_sentence1.append(w1.lower())

            # remove empty strings
            while "" in filtered_sentence1:
                filtered_sentence1.remove("")

            # updating record in col1
            df.at[index1, col1]= ','.join(filtered_sentence1)
        else:
            df.at[index1, col1]= ''

    for index2, description2 in enumerate(df[col2], start_index):
        if not pd.isna(description2):
            word_tokens2 = word_tokenize(description2)

            filtered_sentence2 = []
            for w2 in word_tokens2:
                if not containsLetterAndNumber(w2) and w2 not in stop_words:
                    filtered_sentence2.append(w2.lower())

            # remove empty strings
            while "" in filtered_sentence2:
                filtered_sentence2.remove("")

            # updating record in col2
            df.at[index2, col2]= ','.join(filtered_sentence2)
        else:
            df.at[index2, col2]= ''

    # create 3rd column
    new_column_series = []
    for index3, description3 in enumerate(df[col2], start_index):
        temp1 = df[col1].loc[index3].split(',')

        if not pd.isna(description3):
            temp2 = df[col2].loc[index3].split(',')
            temp3 = temp1 + temp2
            temp3 = set(temp3)

            if '' in temp3:
                temp3.remove('')

            new_column_series.append(list(temp3))
        else:
            new_column_series.append(list(temp1))

    # insert new column
    df.insert(df.shape[1], new_col, new_column_series)

In [155]:
# rename column headers in Excel Avalara file and remove extra leading rows
df_xlsx = df_xlsx.rename(columns={'Unnamed: 0':'AvaTax System Tax Code', 'AvaTax System Tax Codes':'AvaTax System Tax Code Description',
                                 'Unnamed: 2':'Additional AvaTax System Tax Code Information'})
df_xlsx = df_xlsx[2:]

In [156]:
column_name = 'AvaTax System Tax Code Description'
column_name_2 = 'Additional AvaTax System Tax Code Information'
new_col_name = 'tax_parsed'
createParsedColumns(df_xlsx, column_name, column_name_2, 2, new_col_name)
df_xlsx.to_csv("avalara_parsed_2.csv")

df_xlsx

,AvaTax System Tax Code,AvaTax System Tax Code Description,Additional AvaTax System Tax Code Information,tax_parsed
2,D0000000,"digital,goods","digital,goods,generally,viewed,downloadable,it...","[artwork, pdfs, examples, include, generally, ..."
3,D9999999,"temporary,unmapped,digital,goods,sku,taxable,d...","digital,good,mapped,specific,digital,good,tax,...","[sku, reviewed, applicable, limited, entity, u..."
4,DA010000,"digital,goods,advertising,material,digital,phy...","tangible,personal,property,designs,artwork,pho...","[photos, property, artwork, etc, tangible, cus..."
5,DA030000,"digital,goods,art,digital,physical,media","canned,artwork,transferred,customer,electronic...","[electronic, artwork, art, transferred, goods,..."
6,DA040000,"digital,goods,audio,digital,physical,media","audio,music,spoken,word,transferred,customer,e...","[electronic, transferred, goods, customer, wor..."
...,...,...,...,...
2541,SY010200,"yoga,classes,facilities,offer,services,yoga,cl...",,"[classes, yoga, services, facilities, offer]"
2542,SL090200,"lodging,accommodations",,"[accommodations, lodging]"
2543,ST110000,"travel,agency",,"[agency, travel]"
2544,ST112506,"travel,agency,fees",,"[agency, fees, travel]"


In [157]:
# Convert row to header and remove the extra leading rows; also remove extra columns
df_csv = df_csv.rename(columns=df_csv.iloc[8])
df_csv = df_csv.loc[9:]
df_csv = df_csv.drop(columns = ['Version', 'Key', 'Column1', 'Synonym', 'Acronym'])

In [158]:
column_name = 'Segment Title'
column_name_2 = 'Segment Definition'
column_name_3 = 'Family Title'
column_name_4 = 'Family Definition'
column_name_5 = 'Class Title'
column_name_6 = 'Class Definition'
column_name_7 = 'Commodity Title'
column_name_8 = 'Definition'
createParsedColumns(df_csv, column_name, column_name_2, 9, 'segment_parsed')
createParsedColumns(df_csv, column_name_3, column_name_4, 9, 'family_parsed')
createParsedColumns(df_csv, column_name_5, column_name_6, 9, 'class_parsed')
createParsedColumns(df_csv, column_name_7, column_name_8, 9, 'commodity_parsed')

In [159]:
def setDiffKeyWords(df, col1, col2, start_index):
    # col1 is the higher level node
    for index in range(start_index, len(df.index)):
        set_1 = set() if not df[col1].iloc[index] else set(df[col1].iloc[index])
        set_2 = set() if not df[col2].iloc[index] else set(df[col2].iloc[index])
        set_diff = set_2 - set_1
        df.at[index, col2]= list(set_diff)

In [160]:
col_seg = 'segment_parsed'
col_fam = 'family_parsed'
col_cls = 'class_parsed'
col_com = 'commodity_parsed'
setDiffKeyWords(df_csv, col_seg, col_fam, 9)
setDiffKeyWords(df_csv, col_fam, col_cls, 9)
setDiffKeyWords(df_csv, col_cls, col_com, 9)
df_csv.to_csv("unspc_parsed_2.csv")

df_csv

,Segment,Segment Title,Segment Definition,Family,Family Title,Family Definition,Class,Class Title,Class Definition,Commodity,Commodity Title,Definition,segment_parsed,family_parsed,class_parsed,commodity_parsed
9,10000000,"live,plant,animal,material,accessories,supplies","this,segment,includes,live,wild,domesticated,s...",NaN,,,NaN,,,NaN,,,"[includes, segment, materials, live, feeding, ...",[],[livestock],[goats]
10,10000000,"live,plant,animal,material,accessories,supplies","this,segment,includes,live,wild,domesticated,s...",10100000,"live,animals",,NaN,,,NaN,,,"[includes, segment, materials, live, feeding, ...",[],[livestock],[asses]
11,10000000,"live,plant,animal,material,accessories,supplies","this,segment,includes,live,wild,domesticated,s...",10100000,"live,animals",,10101500,livestock,,NaN,,,"[includes, segment, materials, live, feeding, ...",[],[livestock],[mice]
12,10000000,"live,plant,animal,material,accessories,supplies","this,segment,includes,live,wild,domesticated,s...",10100000,"live,animals",,10101500,livestock,,10101501,cats,,"[includes, segment, materials, live, feeding, ...",[],[livestock],[swine]
13,10000000,"live,plant,animal,material,accessories,supplies","this,segment,includes,live,wild,domesticated,s...",10100000,"live,animals",,10101500,livestock,,10101502,dogs,,"[includes, segment, materials, live, feeding, ...",[],[livestock],[rabbits]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
156402,95000000,"land,buildings,structures,thoroughfares","land,waterways,related,real,property,physical,...",95140000.0,"prefabricated,buildings,structures","buildings,structures,used,various,farm,related...",95141900.0,"prefabricated,medical,buildings,structures","buildings,structures,used,medical,purposes,pre...",NaN,,,"[property, buildings, real, thoroughfares, str...","[prefabricated, one, buildings, location, purp...","[prefabricated, buildings, location, medical, ...",[]
156403,95000000,"land,buildings,structures,thoroughfares","land,waterways,related,real,property,physical,...",95140000.0,"prefabricated,buildings,structures","buildings,structures,used,various,farm,related...",95141900.0,"prefabricated,medical,buildings,structures","buildings,structures,used,medical,purposes,pre...",95141901.0,"medical,unit","building,structure,medical,equipment,house,use...","[property, buildings, real, thoroughfares, str...","[prefabricated, one, buildings, location, purp...","[prefabricated, buildings, location, medical, ...","[tests, house, medical, building, used, equipm..."
156404,95000000,"land,buildings,structures,thoroughfares","land,waterways,related,real,property,physical,...",95140000.0,"prefabricated,buildings,structures","buildings,structures,used,various,farm,related...",95141900.0,"prefabricated,medical,buildings,structures","buildings,structures,used,medical,purposes,pre...",95141902.0,"laboratory,unit","building,structure,laboratory,equipment,house,...","[property, buildings, real, thoroughfares, str...","[prefabricated, one, buildings, location, purp...","[prefabricated, buildings, location, medical, ...","[tests, house, building, laboratory, used, equ..."
156405,95000000,"land,buildings,structures,thoroughfares","land,waterways,related,real,property,physical,...",95140000.0,"prefabricated,buildings,structures","buildings,structures,used,various,farm,related...",95141900.0,"prefabricated,medical,buildings,structures","buildings,structures,used,medical,purposes,pre...",95141903.0,"dental,unit","building,structure,dental,equipment,house,used...","[property, buildings, real, thoroughfares, str...","[prefabricated, one, buildings, location, purp...","[prefabricated, buildings, location, medical, ...","[tests, dental, house, building, used, equipme..."


In [168]:
from collections import defaultdict

factory = lambda: defaultdict(factory)

unspc_segments = defaultdict(factory)
unspc_family = defaultdict(factory)
unspc_class = defaultdict(factory)
unspc_commodity = defaultdict(factory)

unspc_family_full = defaultdict(factory)
unspc_class_full = defaultdict(factory)
unspc_commodity_full = defaultdict(factory)

for index in range(9, len(df_csv.index)):
    if not pd.isna(df_csv['Commodity'].iloc[index]):
        seg_key = str(df_csv['Commodity'].iloc[index])[:2]
        fam_key = str(df_csv['Commodity'].iloc[index])[2:4]
        class_key = str(df_csv['Commodity'].iloc[index])[4:6]
        com_key = str(df_csv['Commodity'].iloc[index])[6:8]
    elif not pd.isna(df_csv['Class'].iloc[index]):
        seg_key = str(df_csv['Class'].iloc[index])[:2]
        fam_key = str(df_csv['Class'].iloc[index])[2:4]
        class_key = str(df_csv['Class'].iloc[index])[4:6]
        com_key = ""
    elif not pd.isna(df_csv['Family'].iloc[index]):
        seg_key = str(df_csv['Family'].iloc[index])[:2]
        fam_key = str(df_csv['Family'].iloc[index])[2:4]
        class_key = ""
        com_key = ""
    elif not pd.isna(df_csv['Segment'].iloc[index]):
        seg_key = str(df_csv['Segment'].iloc[index])[:2]
        fam_key = ""
        class_key = ""
        com_key = ""
    else:
        seg_key = ""
        fam_key = ""
        class_key = ""
        com_key = ""

    # segments hash map
    if seg_key:
        if seg_key not in unspc_segments:
            unspc_segments[seg_key] = df_csv['segment_parsed'].iloc[index]

    # family hash map
    if seg_key and fam_key:
        if seg_key not in unspc_family:
            unspc_family[seg_key][fam_key] = df_csv['family_parsed'].iloc[index]
        elif fam_key not in unspc_family[seg_key]:
            unspc_family[seg_key][fam_key] = df_csv['family_parsed'].iloc[index]

        if seg_key + fam_key not in unspc_family_full:
            unspc_family_full[seg_key + fam_key] = df_csv['family_parsed'].iloc[index]

    # class hash map
    if seg_key and fam_key and class_key:
        if seg_key not in unspc_class:
            unspc_class[seg_key][fam_key][class_key] = df_csv['class_parsed'].iloc[index]
        elif fam_key not in unspc_class[seg_key]:
            unspc_class[seg_key][fam_key][class_key] = df_csv['class_parsed'].iloc[index]
        elif class_key not in unspc_class[seg_key][fam_key]:
            unspc_class[seg_key][fam_key][class_key] = df_csv['class_parsed'].iloc[index]

        if seg_key + fam_key + class_key not in unspc_class_full:
            unspc_class_full[seg_key + fam_key + class_key] = df_csv['class_parsed'].iloc[index]

    # commodity hash map
    if seg_key and fam_key and class_key and com_key:
        if seg_key not in unspc_class:
            unspc_commodity[seg_key][fam_key][class_key][com_key] = df_csv['commodity_parsed'].iloc[index]
        elif fam_key not in unspc_commodity[seg_key]:
            unspc_commodity[seg_key][fam_key][class_key][com_key] = df_csv['commodity_parsed'].iloc[index]
        elif class_key not in unspc_commodity[seg_key][fam_key]:
            unspc_commodity[seg_key][fam_key][class_key][com_key] = df_csv['commodity_parsed'].iloc[index]
        elif com_key not in unspc_commodity[seg_key][fam_key][class_key]:
            unspc_commodity[seg_key][fam_key][class_key][com_key] = df_csv['commodity_parsed'].iloc[index]

        if seg_key + fam_key + class_key + com_key not in unspc_class_full:
            unspc_commodity_full[seg_key + fam_key + class_key + com_key] = df_csv['commodity_parsed'].iloc[index]

with open("segments.json", "w") as outfile:
    json.dump(unspc_segments, outfile)

with open("family.json", "w") as outfile:
    json.dump(unspc_family, outfile)

with open("class.json", "w") as outfile:
    json.dump(unspc_class, outfile)

with open("commodity.json", "w") as outfile:
    json.dump(unspc_commodity, outfile)

# print(json.dumps(unspc_segments, sort_keys=True, indent=2))
# print(json.dumps(unspc_family, sort_keys=True, indent=2))
# print(json.dumps(unspc_class, sort_keys=True, indent=2))
# print(json.dumps(unspc_commodity, sort_keys=True, indent=2))

In [ ]:
# unspc_segments
# unspc_family
# unspc_class
# unspc_commodity
new_column_series = []

segment_best_matches = {}
family_best_matches = {}
class_best_matches = {}
commodity_best_matches = {}

for segment_key, segment_keywords in unspc_segments.items():
    for index, keyword_arr_avalara in enumerate((df_xlsx["tax_parsed"])):
        num_matches = len(np.intersect1d(keyword_arr_avalara, segment_keywords))
        if num_matches > 1:
            tax_code = df_xlsx["AvaTax System Tax Code"].iloc[index]
            if segment_key in segment_best_matches:
                segment_best_matches[segment_key].append(tax_code)
            else:
                segment_best_matches[segment_key] = [tax_code]

for family_key, family_keywords in unspc_family_full.items():
    for index, keyword_arr_avalara in enumerate((df_xlsx["tax_parsed"])):
        num_matches = len(np.intersect1d(keyword_arr_avalara, family_keywords))
        if num_matches > 0:
            tax_code = df_xlsx["AvaTax System Tax Code"].iloc[index]
            if family_key in family_best_matches:
                family_best_matches[family_key].append(tax_code)
            else:
                family_best_matches[family_key] = [tax_code]

for class_key, class_keywords in unspc_class_full.items():
    for index, keyword_arr_avalara in enumerate((df_xlsx["tax_parsed"])):
        num_matches = len(np.intersect1d(keyword_arr_avalara, class_keywords))
        if num_matches > 0:
            tax_code = df_xlsx["AvaTax System Tax Code"].iloc[index]
            if class_key in class_best_matches:
                class_best_matches[class_key].append(tax_code)
            else:
                class_best_matches[class_key] = [tax_code]

for commodity_key, commodity_keywords in unspc_commodity_full.items():
    for index, keyword_arr_avalara in enumerate((df_xlsx["tax_parsed"])):
        num_matches = len(np.intersect1d(keyword_arr_avalara, commodity_keywords))
        if num_matches > 0:
            tax_code = df_xlsx["AvaTax System Tax Code"].iloc[index]
            if commodity_key in commodity_best_matches:
                commodity_best_matches[commodity_key].append(tax_code)
            else:
                commodity_best_matches[commodity_key] = [tax_code]

print(segment_best_matches)
print(family_best_matches)
print(class_best_matches)
print(commodity_best_matches)





    # df.insert(df.shape[1], new_col, new_column_series)